#### **1. 设置环境**

导入 Python 标准库、第三方库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import json

# 第三方库
# pip install pandas tqdm
import pandas as pd
from tqdm import tqdm

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
# 大模型句法标注模块
from src.utils import load_multiver_corpus
from src.annotator.syntax_annotation import SyntaxAnnotator

#### **2. 读取语料**

读取包含人工译文和大模型译文的多版本平行语料库

In [7]:
# 指定语料库文件路径 data/output/0_mt_generation.jsonl
# 文件为 JSON 格式
# 包含《鹿鼎记》原文、闵福德译文以及大模型生成译文

data_file = '../data/output/0_mt_generation.jsonl'
print(f"从 {os.path.abspath(data_file)} 文件读取数据 ...")

data = load_multiver_corpus(data_file)
print(f"成功读取 {len(data)} 条平行句对")

# 筛选数据：
# 设置 LIMIT=10，仅标准前 10 条数据
# 设置 LIMIT=None，标注全部数据
LIMIT = 10
#LIMIT = None
if LIMIT:
    data = data.head(LIMIT)
print(f"准备为前 {len(data)} 条数据标注词性/句法结构")

# 预览数据：
# 导入后的数据以 DataFrame 格式存储
# 第一列：原文
# 第二列：多版本译文：
# human：人类译本（闵福德译）
# deepseek-v3.2：deepseek 译本
# qwen3-max：qwen 译本

print("数据前 5 行如下：")
data.head()

从 C:\2026_llm_corpus_annotation\data\output\0_mt_generation.jsonl 文件读取数据 ...
成功读取 305 条平行句对
准备为前 10 条数据标注词性/句法结构
数据前 5 行如下：


,id,source,targets
0,0,北风如刀，满地冰霜。,"{'human': None, 'deepseek-v3.2': 'The north wi..."
1,1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,{'human': 'Along a coastal road somewhere sout...
2,2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,{'human': 'In each of the first three carts a ...
3,3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,{'human': 'The four rear carts were occupied b...
4,4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,{'human': 'The little girl was crying in a con...


#### **3. 加载模型**

加载词性/句法标注模型

In [8]:
# 加载 HanLP 中文模型
# hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH
# 详见：https://hanlp.hankcs.com/docs/api/hanlp/pretrained/mtl.html
# 首次加载需从网络下载模型至本地
# 模型大小：504 MB

print("加载 HanLP 中文模型 ...")
zh_annotator = SyntaxAnnotator(lang='Chinese', device=None)
print(f"成功加载中文模型：{zh_annotator.model_name}\n")

# 加载 HanLP 英文模型
# hanlp.pretrained.mtl.EN_TOK_LEM_POS_NER_SRL_UDEP_SDP_CON_MODERNBERT_BASE
# 详见：https://hanlp.hankcs.com/docs/api/hanlp/pretrained/mtl.html
# 首次加载需从网络下载模型至本地
# 模型大小：620 MB 

print("加载 HanLP 英文模型 ...")
en_annotator = SyntaxAnnotator(lang='English', device=None)
print(f"成功加载英文模型：{en_annotator.model_name}")

加载 HanLP 中文模型 ...


成功加载中文模型：Electra Base

加载 HanLP 英文模型 ...


成功加载英文模型：ModernBERT Base


#### **4. 批量标注**

使用 HanLP 标注中文原文和英文各译本的词汇和句法结构

并以 JSON 格式返回标注结果

In [10]:
# === 核心函数 annotate_data ===
# 调用 HanLP 本地模型批量标注文本，结果保存于 results 列表

# 参数 data：DataFrame 格式的数据（详见步骤 2）
# 参数 zh_annotator：HanLP 中文标注模型接口（详见步骤 3）
# 参数 en_annotator：HanLP 英文标注模型接口（详见步骤 3）

def annotate_data(data, zh_annotator, en_annotator):
    results = []

    # 逐行遍历所有数据
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Syntax Tagging"):
        try:
            # 解析数据
            # 提取汉语原文和各版本译文
            record_id = row['id']
            zh_text = row['source']
            targets_dict = row['targets'] # {'human': '...', 'deepseek-v3.2': '...'}
            
            # 标注中文原文
            if zh_text:
                source_anno = zh_annotator.annotate(zh_text, mode='light')
            else:
                source_anno = None
            
            # 标注英文译文
            target_annos = {}
            
            # 遍历所有译本
            for version_name, en_text in targets_dict.items():
                if en_text and isinstance(en_text, str):
                    # 分别标注各版本译文
                    en_anno = en_annotator.annotate(en_text, mode='light')
                    target_annos[version_name] = en_anno
                else:
                    target_annos[version_name] = None

            # 合并中英文标注结果
            record = {
                "id": f"{record_id:06d}",
                "annotations": {
                    "source_zh": source_anno,
                    "targets_en": target_annos
                }
            }
            results.append(record)
            
        except Exception as e:
            print(f"Error at index {index}: {e}")
            continue
    
    return results

In [11]:
# 注意：
# 本地大模型在 GPU 上的运行速度远快于 CPU
# 因此，HanLP 自动选用 GPU 标注语料

# 可在命令行输入 nvidia-smi，查看是否配备 GPU
# 若电脑未配备 GPU，建议使用免费 GPU 服务器
# 如腾讯云 https://cloud.tencent.com/ 执行本程序

# 使用 NVIDIA GeForce RTX 3050，4GB 显存
# 标注 305 句原文及其 3 个英文译本
# 大约需要 39 秒

# 开始标注
results = annotate_data(data, zh_annotator, en_annotator)

Syntax Tagging: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.64it/s]


#### **5. 保存结果**

将 HanLP 标注结果保存于 JSON 文件

In [12]:
# save_results 函数：
# 将 results 列表转换为 JSON 格式
# 保存于 out_file 文件
def save_results(results, out_file):
     with open(out_file, "wt", encoding="utf-8") as fout:
        for record in results:
            fout.write(json.dumps(record, ensure_ascii=False) + "\n")

In [13]:
# 指定输出文件
if LIMIT:
    out_file = f"../data/output/1_syntax_annotation_limit{LIMIT}.jsonl"
else:
    out_file = f"../data/output/1_syntax_annotation.jsonl"

# 保存标注结果
save_results(results, out_file)
print(f"结果已保存至 {out_file}\n")

结果已保存至 ../data/output/1_syntax_annotation_limit10.jsonl



#### **6. 查看结果**

预览中英文标注结果，查看词性/句法赋码集

In [17]:
# 预览前 3 条数据
for res in results[:3]:
    # 提取数据
    zh_data = res['annotations']['source_zh']
    en_targets = res['annotations']['targets_en']

    print(f"\n[ID]: {res['id']}")
    print("=" * 60)

    # 中文
    # 打印首句句法特征
    zh_text = zh_data['sentences'][0]
    zh_tokens = zh_data['tokens'][0]
    zh_pos = zh_data['pos'][0]
    zh_dep = zh_data['dep'][0]
    print(f"  【原文】:  {zh_text}")
    print(f"   TOK:    {zh_tokens[:5]} ...") 
    print(f"   POS:    {zh_pos[:5]} ...")
    print(f"   DEP:    {zh_dep[:5]} ...")
    print("-" * 60)

    # 英文
    for version, data in en_targets.items():
        if not data: continue
        
        print(f"【{version.upper()}】")
        
        # 打印各版本首句句法特征
        sent_text = data['sentences'][0]
        tokens = data['tokens'][0]
        lemma = data['lem'][0]
        pos = data['pos'][0]
        dep = data['dep'][0]
        
        print(f"   Text:   {sent_text}")
        print(f"   TOK:    {tokens[:5]} ...")
        print(f"   LEM:    {lemma[:5]} ...")
        print(f"   POS:    {pos[:5]} ...")
        print(f"   DEP:    {dep[:5]} ...")
        
        print("." * 30)


[ID]: 000000
  【原文】:  北风如刀，满地冰霜。
   TOK:    ['北风', '如', '刀', '，', '满地'] ...
   POS:    ['n', 'v', 'n', 'w', 'n'] ...
   DEP:    [(2, 'nsubj'), (0, 'root'), (2, 'dobj'), (2, 'punct'), (6, 'advmod')] ...
------------------------------------------------------------
【DEEPSEEK-V3.2】
   Text:   The north wind cut like a knife, and frost covered the ground.
   TOK:    ['The', 'north', 'wind', 'cut', 'like'] ...
   LEM:    ['the', 'north', 'wind', 'cut', 'like'] ...
   POS:    ['DT', 'NN', 'NN', 'VBD', 'IN'] ...
   DEP:    [(3, 'det'), (3, 'compound'), (4, 'nsubj'), (0, 'root'), (7, 'case')] ...
..............................
【QWEN3-MAX】
   Text:   The north wind cuts like a knife; ice and frost cover the ground.
   TOK:    ['The', 'north', 'wind', 'cuts', 'like'] ...
   LEM:    ['the', 'north', 'wind', 'cut', 'like'] ...
   POS:    ['DT', 'NN', 'NN', 'VBZ', 'IN'] ...
   DEP:    [(3, 'det'), (3, 'compound'), (4, 'nsubj'), (0, 'root'), (7, 'case')] ...
..............................

[ID]: 000